# 0. DataSetting

In [1]:
#import packages & libraries
import pandas as pd
import os
import glob
from sklearn.model_selection import StratifiedKFold
from scipy.ndimage import rotate
import numpy as np
import pydicom
from skimage import measure
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
import h5py

In [2]:
import torch

# GPU 사용 가능 여부 확인
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU 가속 사용 가능")
else:
    device = torch.device("cpu")
    print("GPU 가속 사용 불가능")

# 간단한 연산을 GPU에서 실행
a = torch.tensor([1.0, 2.0, 3.0], device=device)
b = torch.tensor([4.0, 5.0, 6.0], device=device)
c = a + b

print("연산 결과:", c)

GPU 가속 사용 가능
연산 결과: tensor([5., 7., 9.], device='cuda:0')


In [2]:
num_cores = os.cpu_count()
print(f"Number of available cores: {num_cores}")

Number of available cores: 48


In [3]:
num_gpus = torch.cuda.device_count()
print(f"Number of available GPUs: {num_gpus}")

NameError: name 'torch' is not defined

## 0-1 Ground Truh Label Setting

In [3]:
# 주 데이터 폴더 경로
main_data_folder = "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast"
image_data_folder = "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/101~200/101~200_screen"

# 세부 폴더의 이름
sub_folder_name = '101~200'

# 개별 환자 폴더 설정
patient_folder_names = [f for f in os.listdir(image_data_folder) if os.path.isdir(os.path.join(image_data_folder, f))]

# Find the first Excel file in the directory
excel_file_path = glob.glob(f"{main_data_folder}/{sub_folder_name}/*.xlsx")[0]
excel_data = pd.read_excel(excel_file_path)

labels = excel_data['Molecular subtype(1 : luminal A, 2 : luminal B, 3: Her2 positive, 4: Triple negative)'].values

## 0-2 Roi Dataset Setting

In [4]:
# 병변을 나타내는 색상들의 RGB 값
lesion_colors = np.array([
    [255, 0, 0],      # Red
    [255, 255, 0],    # Yellow1
    [255, 102, 0],    # Yellow2
    [0, 102, 255],   # Blue1
    [0, 0, 255],     # Blue2
    [0, 255, 0],    # Green
    [128, 0, 255]   # Purple
])

# DICOM 파일들을 읽고 이미지 데이터를 가져오는 함수
def load_patient_images(patient_folder):
    
    # patient_folder 내의 DICOM 파일들을 가져오기
    dicom_files = [os.path.join(patient_folder, f) for f in os.listdir(patient_folder) if f.endswith('.dcm')]

    # DICOM 파일들을 읽기
    dicoms = [pydicom.dcmread(d) for d in dicom_files]

    # 각 DICOM 파일의 이미지 위치 정보를 기준으로 정렬
    dicoms.sort(key=lambda d: float(d.ImagePositionPatient[2]))

    # 각 DICOM 파일의 pixel_array를 가져오기
    images = np.stack([d.pixel_array for d in dicoms])

    return images

# 병변 영역을 찾는 함수
def find_lesions(images):
    lesion_mask = np.any([np.all(images == color, axis=-1) for color in lesion_colors], axis=0)
    return lesion_mask

# 가장 큰 병변의 Bounding Volume을 찾는 함수
def find_largest_lesion_and_bounding_volume(lesion_mask):
    
    # 3D Connected Component Labeling 수행
    labels = measure.label(lesion_mask)

    # 각 병변의 크기를 계산하고 가장 큰 병변을 찾기
    sizes = np.bincount(labels.ravel())
    max_label = sizes[1:].argmax() + 1  # 0 레이블은 배경이므로 제외

    # 가장 큰 병변에 해당하는 픽셀들의 좌표를 찾기
    lesion_pixels = np.where(labels == max_label)

    # bounding volume의 모서리를 찾기
    min_corner = np.array([np.min(i) for i in lesion_pixels])
    max_corner = np.array([np.max(i) for i in lesion_pixels])

    return min_corner, max_corner

# DICOM 파일들을 읽고 이미지 데이터를 가져오는 함수
def load_dicoms(patient_folder):
    
    # patient_folder 내의 DICOM 파일들을 가져오기
    dicom_files = [os.path.join(patient_folder, f) for f in os.listdir(patient_folder) if f.endswith('.dcm')]

    # DICOM 파일들을 읽기
    dicoms = [pydicom.dcmread(d) for d in dicom_files]

    # 각 DICOM 파일의 이미지 위치 정보를 기준으로 정렬
    dicoms.sort(key=lambda d: float(d.ImagePositionPatient[2]))

    # 각 DICOM 파일의 pixel_array를 가져오기
    images = np.stack([d.pixel_array for d in dicoms])

    return images, dicoms

# images를 3D 복셀로 변환합니다.
def create_voxel(images, dicoms):
    
    # 모든 2D DICOM 이미지를 하나의 3D numpy 배열로 결합
    voxel = images.astype(float)

    # DICOM 파일의 RescaleIntercept와 RescaleSlope를 적용하여 픽셀 값 조정
    voxel *= np.float64(dicoms[0].RescaleSlope)
    voxel += np.float64(dicoms[0].RescaleIntercept)

    return voxel

In [5]:
def process_patient_folder(patient_folder_name):
    
    patient_folder_path = os.path.join(image_data_folder, patient_folder_name) # 환자 폴더 경로
    folder_list = os.listdir(patient_folder_path)
    
    # 환자의 일련번호를 patient_num 변수에 저장
    patient_num = int(patient_folder_name)

    # 'cad'와 'precontrast' 폴더 찾기
    cad_folder_name = next(f for f in folder_list if 'cad' in f.lower())
    precontrast_folder_name = next(f for f in folder_list if 'precontrast' in f.lower())

    # 경로 생성
    cad_folder = os.path.join(patient_folder_path, cad_folder_name)
    precontrast_folder = os.path.join(patient_folder_path, precontrast_folder_name)

    # 각 폴더의 하위 폴더 찾기
    cad_subfolder_name = next(f for f in os.listdir(cad_folder) if os.path.isdir(os.path.join(cad_folder, f)))
    precontrast_subfolder_name = next(f for f in os.listdir(precontrast_folder) if os.path.isdir(os.path.join(precontrast_folder, f)))

    # DICOM 폴더 경로 생성
    dicom_folder = os.path.join(cad_folder, cad_subfolder_name)
    dicom_folder_pre = os.path.join(precontrast_folder, precontrast_subfolder_name)

    # DICOM 이미지들을 로드하기
    images = load_patient_images(dicom_folder)
    
    # 병변 마스크를 생성하기
    lesion_mask = find_lesions(images)
    
    # 가장 큰 병변의 Bounding Volume 찾기
    min_corner, max_corner = find_largest_lesion_and_bounding_volume(lesion_mask)
    
    # DICOM 이미지를 로드하기
    images_pre, dicoms_pre = load_dicoms(dicom_folder_pre)
    
    # images_pre를 3D 복셀로 변환합니다.
    voxels_pre = create_voxel(images_pre, dicoms_pre)

    # precontrast MRI에서 해당 ROI 잘라내기
    roi_precontrast = voxels_pre[min_corner[0]:max_corner[0], min_corner[1]:max_corner[1], min_corner[2]:max_corner[2]]

    return patient_num, roi_precontrast

# HDF5 파일로 저장하기 위한 h5py.File 인스턴스 생성
save_folder = '/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/101~200/hdf5_101-200' # 저장될 폴더 경로 설정
hdf5_file_name = f'{sub_folder_name}_data.h5' # 저장될 HDF5 파일 이름 설정

with h5py.File(os.path.join(save_folder, hdf5_file_name), 'w') as hdf5_file:
    with ProcessPoolExecutor(max_workers=30) as executor:
        for patient_num, roi_precontrast in executor.map(process_patient_folder, patient_folder_names):
                                 
            # roi_precontrast를 해당 환자의 번호로 HDF5 파일에 저장
            hdf5_file.create_dataset(str(patient_num), data=roi_precontrast, chunks=True)

In [6]:
import h5py

# 저장된 HDF5 파일 열기
with h5py.File(os.path.join(save_folder, hdf5_file_name), 'r') as hdf5_file:
   
    # 파일 내에 저장된 모든 데이터셋의 이름 출력
    for key in hdf5_file.keys():
        print(key)
        
        # 각 데이터셋의 shape 출력
        print(hdf5_file[key].shape)

101
(112, 274, 266)
102
(23, 26, 30)
103
(17, 34, 33)
104
(120, 118, 355)
105
(104, 135, 176)
106
(38, 67, 65)
107
(43, 48, 100)
108
(84, 40, 94)
109
(88, 178, 165)
110
(94, 40, 21)
111
(137, 187, 158)
112
(17, 38, 33)
113
(135, 102, 77)
114
(77, 88, 60)
115
(107, 59, 151)
116
(135, 176, 187)
117
(88, 30, 100)
118
(39, 42, 72)
119
(57, 30, 118)
120
(48, 98, 57)
121
(32, 28, 45)
122
(12, 20, 36)
123
(35, 35, 86)
124
(40, 37, 82)
125
(25, 57, 42)
126
(35, 48, 45)
127
(103, 166, 176)
128
(53, 34, 83)
129
(66, 74, 49)
130
(43, 34, 68)
131
(55, 31, 96)
132
(30, 69, 49)
133
(66, 116, 173)
134
(63, 65, 113)
135
(75, 100, 192)
136
(19, 53, 37)
137
(46, 41, 107)
138
(135, 201, 357)
139
(91, 67, 62)
140
(134, 189, 204)
141
(13, 25, 20)
142
(107, 157, 93)
143
(105, 139, 240)
144
(31, 33, 39)
145
(58, 34, 106)
146
(82, 84, 134)
147
(111, 12, 23)
148
(121, 162, 169)
149
(86, 175, 108)
150
(142, 57, 100)
151
(85, 14, 23)
152
(102, 183, 82)
153
(118, 149, 204)
154
(33, 132, 44)
155
(26, 48, 70)
156
(